In [1]:
import numpy as np
import pandas as pd
from env import MufexKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import *
from quantfreedom.simulate import run_df_backtest, or_backtest
from quantfreedom.strategies.ex_strat_02 import SimpleBreakoutDynamicLookback


mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [2]:
candles = mufex_main.get_candles(
    symbol="BTCUSDT",
    timeframe="5m",
    since_timestamp=1697328000000,
    until_timestamp=1698105600000,
)

In [3]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)

In [4]:
# backtest_settings = BacktestSettings(
#     array_size=200,
#     gains_pct_filter=-90,
#     total_trade_filter=10,
#     upside_filter=-np.inf,
# )
backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([6]),
    max_trades=np.array([5]),
    account_pct_risk_per_trade=np.array([3]),
    risk_reward=np.array([6]),
    sl_based_on_add_pct=np.array([0.1]),
    sl_based_on_lookback=np.array([20]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([1]),
    trail_sl_when_pct=np.array([1]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=10,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

In [5]:
strategy = SimpleBreakoutDynamicLookback(
    volitility_lb=np.array([30, 50]),
    max_high_lb=np.array([20]),
    long_short="long",
)

In [6]:
run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    static_os=static_os,
    strategy=strategy,
    logger_bool=True,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 2
Total order settings to test: 1
Total combinations of settings to test: 2
Total candles: 2,592
Total candles to test: 5,184


,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,qf_score,fees_paid,total_pnl,ending_eq
0,0,0,15.0,10,5,93.347,66.667,0.824,240.599,933.467,1933.467
1,1,0,16.0,10,6,78.170,62.500,0.626,279.921,781.699,1781.699


In [7]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    static_os=static_os,
    strategy=strategy,
    dos_index=0,
    ind_set_index=0,
    plot_results=True,
    logger_bool=False,
)